In [ ]:
!git clone https://github.com/Kuma221203/CS114.N21

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import os
import cv2
import shutil
from keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn import neighbors, metrics, svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras import activations
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Quang task
# Đường dẫn đến file txt chứa danh sách URL cần clone
file_path = 'C:\\Users\\quang\\Desktop\\cs114\\pathgit.txt'

# Kiểm tra xem file tồn tại hay không
if not os.path.isfile(file_path):
    print("File không tồn tại.")
    exit()

# Đọc nội dung của file
with open(file_path, "r") as file:
    lines = file.readlines()

# Loại bỏ ký tự newline ('\n') khỏi mỗi dòng
lines = [line.strip() for line in lines]
lines
# Thực hiện git clone cho từng dòng trong file
# for line in lines:
#     os.system("git clone " + line)

def rename_images(folder_path, new_name_prefix):
    # Lặp qua tất cả các file trong thư mục
    for filename in os.listdir(folder_path):
        # Kiểm tra nếu đây là file ảnh
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Tạo đường dẫn cũ và mới cho file
            old_path = os.path.join(folder_path, filename)
            new_name = new_name_prefix + '_' + filename
            new_path = os.path.join(folder_path, new_name)
            
            # Đổi tên file
            os.rename(old_path, new_path)
            
            # In ra thông báo sau khi đổi tên thành công
            # print(f"Đã đổi tên file {filename} thành {new_name}")
def movefile(source_folder,destination_folder):
    # Lặp qua tất cả các file trong thư mục gốc
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)  # Đường dẫn đầy đủ của file gốc
        # destination_path = os.path.join(destination_folder, filename)  # Đường dẫn đầy đủ của file đích
        shutil.move(source_path, destination_folder)  # Chuyển file
        # print(source_path,destination_folder)
pathde="C:\\Users\\quang\\Desktop\\cs114\\Datasets\\HandwrittenDigitDataset\\hand_written_digits\\"
for i in range(0,10):
    rename_images(pathde+str(i),"14")
for i in range(0,10):
    movefile(pathde+str(i),"C:\\Users\\quang\\Desktop\\cs114\\Datasets\\"+str(i))

In [ ]:
#Quan task

In [3]:
X = []
labels = []
kernel = np.ones((5, 5), np.uint8)
path = "/content/CS114.N21/Dataset"

imgPaths = []

for name in os.listdir(path):
    path_numbers = path + "/" + name
    for img_name in os.listdir(path_numbers):
        imgPath = path_numbers + "/" + img_name
        imgPaths.append(imgPath)
        img_root = cv2.imread(imgPath)
        img_root = cv2.resize(img_root, (280, 280))
        lab = cv2.cvtColor(img_root, cv2.COLOR_BGR2Lab)
        l_channel, a, b = cv2.split(lab)
        clade = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl = clade.apply(l_channel)
        limg = cv2.merge((cl,a,b))
        img = cv2.cvtColor(limg, cv2.COLOR_BGR2GRAY)
        _, img_gray = cv2.threshold(img, 125, 257, cv2.THRESH_BINARY_INV)
        img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_CLOSE, kernel, iterations = 5)
        contours, _ = cv2.findContours(img_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_contour = [cv2.contourArea(cnt) for cnt in contours]
        area_sort = np.argsort(area_contour)[::-1]
        x, y, w, h = cv2.boundingRect(contours[area_sort[0]])
        pre_image = img_gray[y:(y+h), x:(x+w)]
        pre_image = cv2.resize(pre_image, (25, 25))
        pre_image = cv2.copyMakeBorder(pre_image,3,3,3,3,cv2.BORDER_CONSTANT, value = 0)
        img_reshape = cv2.resize(pre_image, (28, 28))
        X_number = np.array(img_reshape).reshape((784))
        X.append(X_number)
        labels.append(int(name))
X = np.array(X)
y = labels
y = np.array(y)
print(X.shape)
print(y.shape)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
X_imgPath_train, X_imgPath_test, y_train, y_test = train_test_split(list(zip(X, imgPaths)), y, test_size = 0.3333)
X_train = np.array([x_imgPath[0] for x_imgPath in X_imgPath_train])
X_test = np.array([x_imgPath[0] for x_imgPath in X_imgPath_test])
imgPath_test = np.array([x_imgPath[1] for x_imgPath in X_imgPath_test])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.142857)

In [ ]:
print(X_train.shape, X_test.shape, X_val.shape)
size_train, _ = X_train.shape
size_val, _ = X_val.shape
size_test, _ = X_test.shape

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 2)
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_predict))
print(y_test)
print(y_predict)

In [ ]:
X_train = np.reshape(X_train, (size_train, 28, 28, 1))
X_val = np.reshape(X_val, (size_val, 28, 28, 1))
X_test = np.reshape(X_test, (size_test, 28, 28, 1))
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test_cnn = to_categorical(y_test)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5,5), activation = activations.relu, padding = 'same', input_shape = (28, 28, 1)))
model.add(Conv2D(32, (5,5), activation = activations.relu, padding = 'same', input_shape = (28, 28, 1)))
model.add(MaxPooling2D((2,2), strides = 2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation = activations.relu, padding = 'same', input_shape = (28, 28, 1)))
model.add(Conv2D(64, (3,3), activation = activations.relu, padding = 'same', input_shape = (28, 28, 1)))
model.add(MaxPooling2D((2,2), strides = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = activations.relu))
model.add(Dropout(0.25))
model.add(Dense(10, activation = activations.softmax))

model.summary()
# fit model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')
epoch = 30
batch_size = 112

In [ ]:
#checkpoint
filepath="/content/best_checkpoint"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
# fit model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')
epoch = 30
batch_size = 112
model.fit(x = X_train, y = y_train, batch_size = batch_size, epochs = epoch, callbacks = [], validation_data = (X_val, y_val))

In [ ]:
y_predict_raw = model.predict(X_test)
y_predict = np.argmax(y_predict_raw, axis = 1)
print(y_predict.shape, y_test.shape)
print(metrics.accuracy_score(y_test, y_predict))

# Confusion matrix

In [ ]:
print(
    f"Classification report for classifier {clf}:\n"
    f"{metrics.classification_report(y_test, y_predict)}\n"
)

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_predict)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

# Chart

In [ ]:
import matplotlib.pyplot as plt

# Dữ liệu về tỉ lệ đúng/sai của các cột
correct = [len([i for i, y in enumerate(y_test) if y == num and y == y_predict[i]]) for num in range(10)]
incorrect = [len([i for i, y in enumerate(y_test) if y == num and y != y_predict[i]]) for num in range(10)]

correctRatio = [cor / (cor + incor) for cor, incor in zip(correct, incorrect)]
incorrectRatio = [1 - cor for cor in correctRatio]

# Labels cho các cột
# labels = ['Cột 1', 'Cột 2', 'Cột 3', 'Cột 4', 'Cột 5']
labels = list(range(10))

# Tạo một Figure và Axes
fig, ax = plt.subplots()

# Vẽ biểu đồ cột ngang
ax.barh(labels, correctRatio, label='Đúng', color='green')
ax.barh(labels, incorrectRatio, left=correctRatio, label='Sai', color='red')

# Đặt tiêu đề và nhãn trục
ax.set_title('Tỉ lệ đúng/sai của các chữ số')
ax.set_xlabel('Tỉ lệ')
ax.set_ylabel('Chữ số')

# Hiển thị chú thích
ax.legend()

# Hiển thị biểu đồ
plt.show()

# Kiểm tra kết quả sai

In [ ]:
from PIL import Image

In [1]:
# Hiển thị các ảnh bị dự đoán sai kèm theo nhãn dự đoán
rowShow = 3 
figWidthShow = 18
subFigHeightShow = 3

def showRawImg_ExtractedData_Labels(rawImgs, extractedDatas, actLabels, predLabels):
    itemCount = len(actLabels)

    rowCount = itemCount // rowShow + 1 if itemCount % rowShow else itemCount // rowShow
    figSizeShow = (figWidthShow, subFigHeightShow * rowCount) # số đầu là chiều ngang, sau: chiều dọc



    fig, axes = plt.subplots( itemCount // rowShow + 1 if itemCount % rowShow else itemCount // rowShow, rowShow * 3, figsize=figSizeShow)
    # fig, axes = plt.subplots( itemCount // rowShow + 1 if itemCount % rowShow else itemCount // rowShow, rowShow * 3)
    print(rowShow * 3, itemCount // rowShow + 1 if itemCount % rowShow else itemCount // rowShow)
    # rawImgs, extractedDatas, labels
    for i in range(itemCount):
        _x = i // rowShow # số hàng
        _y = i % rowShow # số cột

        # show raw img
        ax = axes[_x, _y * 3]
        # ax.imshow(rawImgs[i], cmap='gray')
        img = Image.open(rawImgs[i])
        ax.imshow(img)
        # ax.imshow(rawImgs[i], cmap='gray', aspect='auto')

        # show extracted img
        ax = axes[_x, _y * 3 + 1]
        ax.imshow(extractedDatas[i], cmap='gray')
        # show label
        ax = axes[_x, _y * 3 + 2]
        # ax.text(0.5, 0.5, 'Predicted: {} \n Actual: {}'.format(actLabels[i], predLabels[i]), ha='left', va='center', fontsize=20)
        ax.text(0.5, 0.5, f'Actual: {actLabels[i]} \n Predicted: {predLabels[i]}', ha='center', va='center', fontsize=20)
        ax.axis('off')
    # xoá mấy cái cạnh ở cuối
    for i in range(itemCount % rowShow, rowShow):
        _y = i % rowShow # số cột
        for j in range(3):
            ax = axes[rowCount - 1, _y * 3 + j]
            ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def showRawImg_ExtractedData_Labels_N(rawImgs, extractedDatas, actLabels, predLabels, index):
    showRawImg_ExtractedData_Labels(
        rawImgs[index], 
        extractedDatas[index], 
        actLabels[index],
        predLabels[index])

In [ ]:
misclassified_indices = np.where(y_predict != y_test)[0]
print(f"Số lượng dự đoán sai: {len(misclassified_indices)}/{len(y_predict)}")

In [ ]:
_X_test = X_test.reshape((size_test, 28, 28))

# showRawImg_ExtractedData_Labels_N(_X_test, _X_test, y_test, y_predict, misclassified_indices[10:50])
showRawImg_ExtractedData_Labels_N(imgPath_test, _X_test, y_test, y_predict, misclassified_indices[10:50])